In [3]:
import torch
import torch.nn as nn
import numpy as np 
import matplotlib.pyplot as plt 
from torch.nn import functional as F

In [5]:
# read the input to inspect 
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(f'{len(text)/1000000 : .2f}M')

 1.12M


In [6]:
# vocab which are unique chars in the input text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars), type(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz <class 'list'>
65


In [7]:
# tokenize 
stoi = {s:i for i, s in enumerate(chars)}
itos = {i:s for s, i in stoi.items()}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[n] for n in l])

print(f'encode={encode("hi There!")}')
print(f'decode={decode([46, 47, 1, 32, 46, 43, 56, 43, 2])}')

encode=[46, 47, 1, 32, 46, 43, 56, 43, 2]
decode=hi There!


In [8]:
# lets encode the entire dataset and store it into a torch.Tensor
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])
print(decode(data[:100].numpy()))

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [9]:
# lets split dataset into train and validation set
n = int(0.9 * len(data)) # first 90% will be train, and rest val set
train_data = data[:n]
val_data = data[n:]

train_data.shape, val_data.shape

(torch.Size([1003854]), torch.Size([111540]))

In [10]:
block_size = 8  # also called time dimension T
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'when input is {context} the target: {target}')

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [12]:
# batch B
torch.manual_seed(1337)
batch_size = 4 # how many  independent sequences will be process in parallel ?
block_size = 8 # what is the max context length for predictions ?

def get_batch(split):
    # generate a small batch of data of inputs x and y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(0, len(data) - block_size, (batch_size,))  # gives random 4(batch_size) numbers 
    x = torch.stack([data[i:i+block_size] for i in ix]) # for each i pluck out 8(block_size) sequence of numbers starts from that  i and stack them
    y = torch.stack([data[i+1:i+block_size+1] for i in ix]) # do same as above but offset by 1 for target
    return x, y

xb, yb = get_batch('train')
print('inputs xb')
print(xb.shape)
print(xb)
print('targets yb:')
print(yb.shape)
print(yb)

print('-----------------------')
'''
for b in range(batch_size):  # 4
    for t in range(block_size): # 8
        context = xb[b, :t+1]
        target = yb[b, t]
        print(f'when input is {context.tolist()} the target is {target.tolist()}')
'''

inputs xb
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets yb:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-----------------------


"\nfor b in range(batch_size):  # 4\n    for t in range(block_size): # 8\n        context = xb[b, :t+1]\n        target = yb[b, t]\n        print(f'when input is {context.tolist()} the target is {target.tolist()}')\n"

In [13]:
# bigram language model
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B, T, C)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get predictions
            logits, loss = self.forward(idx)
            # focus on the last step of dim T,which contains next idx logits
            logits = logits[:, -1, :] # B, T, C becomes B, C
            # apple softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # B, C
            # sample distribution
            idx_next = torch.multinomial(probs, num_samples=1) # B, 1
            # append sampled index to the running sequence 
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape, loss.item())

idx = torch.zeros((1, 1), dtype=torch.long) # B = 1, T = 1 basically 0 kick off the generation
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([32, 65]) 4.878634929656982

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [14]:
# optimizer 
optimizer = torch.optim.AdamW(params=m.parameters(), lr=1e-3)

In [15]:
batch_size = 32
for steps in range(10000):
    # sample a batch of data
    xb, yb = get_batch('train')
    # forward pass and evaluate loss
    logits, loss = m(xb, yb)
    # optimizer zero grad
    optimizer.zero_grad()
    # loss backward
    loss.backward()
    # update params
    optimizer.step()

print(loss.item())


2.5727508068084717


In [16]:
idx = torch.zeros((1, 1), dtype=torch.long) # B = 1, T = 1 basically 0 kick off the generation
print(decode(m.generate(idx, max_new_tokens=300)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht 


### The mathematical trick in self-attention

In [17]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2  # Batch, Time, Channels
x = torch.randn(size=(B, T, C))
print(x.shape)
print(x[0])

torch.Size([4, 8, 2])
tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9211,  1.5433]])


In [18]:
#  version 1
# we want x[b, t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1]   # (t, C)
        xbow[b, t] = torch.mean(xprev, dim=0)

In [19]:
# version 2
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(dim=1, keepdim=True)
xbow2 = wei @ x     # (B, T, T) @ (B, T, C)  --------> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [20]:
# version 3: softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow2, xbow3)

True

In [26]:
# version 4: self-attention!
torch.manual_seed(1337)
B, T, C = 4, 8, 32 # batch, time, channels
x = torch.randn(B, T, C)

# lets see a single head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False )
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)

wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ----> (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei / wei.sum(dim=1, keepdim = True)
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
# out = wei @ x
out.shape

torch.Size([4, 8, 16])

In [27]:
wei[0]

tensor([[1.1666e-01, 3.2651e-01, 6.4770e-02, 2.3994e-01, 6.2988e-03, 3.1155e-03,
         1.7512e-01, 6.7580e-02],
        [1.0867e-01, 3.4895e-01, 6.5705e-02, 3.1090e-01, 2.8814e-04, 1.1608e-02,
         6.7893e-02, 8.5992e-02],
        [5.7723e-02, 1.7120e-01, 4.8102e-02, 4.2670e-02, 4.2339e-03, 5.9730e-01,
         5.2765e-02, 2.6007e-02],
        [6.5622e-02, 1.5809e-01, 8.9974e-02, 5.0016e-02, 1.7745e-02, 5.4906e-01,
         3.3824e-02, 3.5661e-02],
        [6.6895e-03, 5.5707e-03, 9.3329e-03, 3.1703e-03, 9.5610e-01, 1.2756e-03,
         7.0703e-03, 1.0791e-02],
        [2.2494e-04, 2.0797e-05, 2.3152e-04, 1.3938e-04, 9.8151e-01, 1.6868e-02,
         5.0018e-04, 5.0369e-04],
        [8.4295e-02, 1.4484e-02, 1.1474e-01, 8.4595e-02, 4.5058e-01, 1.0997e-02,
         7.0262e-02, 1.7005e-01],
        [1.0884e-01, 1.2810e-01, 1.4517e-01, 1.1139e-01, 1.1497e-02, 2.3622e-01,
         1.2572e-01, 1.3307e-01]], grad_fn=<SelectBackward0>)